In [2]:
import numpy as np
import queue
import copy
import matplotlib.pyplot as plt

### MM1 Queue

In [2]:
import simpy
import random

class MM1Queue:
    def __init__(self, env, arrival_rate, service_rate):
        # Initialize the simulation environment, server resource, and parameters
        self.env = env
        self.server = simpy.Resource(env, capacity=1)
        self.arrival_rate = arrival_rate
        self.service_rate = service_rate

    def arrival_process(self):
        # Generates customers following a Poisson process and triggers service process
        customer_id = 1
        while True:
            # Generate inter-arrival time based on Poisson process
            inter_arrival_time = random.expovariate(self.arrival_rate)
            yield self.env.timeout(inter_arrival_time)

            # Start the service process for the arriving customer
            self.env.process(self.service_process(customer_id))
            customer_id += 1

    def service_process(self, customer_id):
        # Processes the service for a customer
        with self.server.request() as request:
            # Request the server for service
            yield request

            # Generate service time based on exponential distribution
            service_time = random.expovariate(self.service_rate)
            yield self.env.timeout(service_time)

            # Customer has been served, print the details
            print(f"Customer {customer_id} served at time {self.env.now}")

def mm1_simulation(arrival_rate, service_rate, simulation_time):
    # Run the MM1 queue simulation
    env = simpy.Environment()
    mm1_queue = MM1Queue(env, arrival_rate, service_rate)
    
    # Start the arrival process
    env.process(mm1_queue.arrival_process())
    
    # Run the simulation until the specified time
    env.run(until=simulation_time)

if __name__ == "__main__":
    # Adjust these parameters based on your requirements
    arrival_rate = 0.2
    service_rate = 0.25
    simulation_time = 10

    # Run the MM1 queue simulation
    mm1_simulation(arrival_rate, service_rate, simulation_time)


Customer 1 served at time 2.2831886909106527


### MMC Queue

In [2]:
import simpy
import random

class MMCQueue:
    def __init__(self, env, num_servers, arrival_rate, service_rate):
        # Initialize the MMCQueue object with simulation environment, number of servers, arrival rate, and service rate
        self.env = env
        self.servers = simpy.Resource(env, capacity=num_servers)
        self.arrival_rate = arrival_rate
        self.service_rate = service_rate
        self.waiting_time = 0
        self.num_served_customers = 0

    def customer_arrival_process(self):
        # Process representing the arrival of customers to the system
        customer_id = 1
        while True:
            # Generate exponentially distributed inter-arrival times
            inter_arrival_time = random.expovariate(self.arrival_rate)
            yield self.env.timeout(inter_arrival_time)

            # Initiate the service process for the arrived customer
            self.env.process(self.customer_service_process(customer_id))
            customer_id += 1

    def customer_service_process(self, customer_id):
        # Process representing the service of customers by servers
        arrival_time = self.env.now
        with self.servers.request() as request:
            yield request
            # Generate exponentially distributed service times
            service_time = random.expovariate(self.service_rate)
            yield self.env.timeout(service_time)

            # Calculate waiting time for the served customer
            self.waiting_time += self.env.now - arrival_time
            self.num_served_customers += 1

    def run_simulation(self, sim_time):
        # Run the simulation for a specified time
        self.env.process(self.customer_arrival_process())
        self.env.run(until=sim_time)

if __name__ == "__main__":
    num_servers = 2  # Number of servers in the system
    arrival_rate = 0.2  # Customer arrival rate
    service_rate = 0.3  # Service rate
    simulation_time = 50  # Total simulation time

    env = simpy.Environment()
    mmc_queue = MMCQueue(env, num_servers, arrival_rate, service_rate)

    # Run the MMCQueue simulation
    mmc_queue.run_simulation(simulation_time)

    # Calculate and print the average waiting time for served customers
    average_waiting_time = mmc_queue.waiting_time / mmc_queue.num_served_customers
    print(f"Average Waiting Time: {average_waiting_time:.2f} time units")


Average Waiting Time: 1.79 time units


### MG1 Queue

In [1]:
import simpy
import numpy as np

class MG1Queue:
    def __init__(self, env, arrival_rate, service_rate):
        self.env = env
        self.server = simpy.Resource(env, capacity=1)
        self.arrival_rate = arrival_rate
        self.service_rate = service_rate
        self.queue = simpy.Store(env)
        self.env.process(self.customer_generator())

    def customer_generator(self):
        customer_id = 0
        while True:
            inter_arrival_time = np.random.exponential(1/self.arrival_rate)
            yield self.env.timeout(inter_arrival_time)
            customer_id += 1
            self.env.process(self.customer(customer_id))

    def customer(self, customer_id):
        arrival_time = self.env.now
        with self.server.request() as request:
            yield request
            service_time = np.random.exponential(1/self.service_rate)
            yield self.env.timeout(service_time)
            departure_time = self.env.now
            waiting_time = departure_time - arrival_time
            print(f"Customer {customer_id} served. Waiting time: {waiting_time}")

def main():
    arrival_rate = 0.5 
    service_rate = 0.8 
    simulation_time = 10 

    env = simpy.Environment()
    mg1_queue = MG1Queue(env, arrival_rate, service_rate)

    env.run(until=simulation_time)

if __name__ == "__main__":
    main()


Customer 1 served. Waiting time: 1.816247526918484
Customer 2 served. Waiting time: 1.5660577176978427
Customer 3 served. Waiting time: 4.431049112819595
Customer 4 served. Waiting time: 4.494157094756419


In [6]:
import simpy

class MD1Queue:
    def __init__(self, env, arrival_interval, service_time):
        self.env = env
        self.server = simpy.Resource(env, capacity=1)
        self.arrival_interval = arrival_interval
        self.service_time = service_time
        self.customer_id = 0
        self.env.process(self.customer_generator())

    def customer_generator(self):
        while True:
            yield self.env.timeout(self.arrival_interval)
            self.customer_id += 1
            self.env.process(self.customer(self.customer_id))

    def customer(self, customer_id):
        arrival_time = self.env.now
        with self.server.request() as request:
            yield request
            service_start_time = self.env.now
            yield self.env.timeout(self.service_time)
            departure_time = self.env.now
            waiting_time = service_start_time - arrival_time
            print(f"Customer {customer_id} served. Waiting time: {waiting_time}")

def main():
    arrival_interval = 2  
    service_time = 1  
    simulation_time = 10 

    env = simpy.Environment()
    md1_queue = MD1Queue(env, arrival_interval, service_time)

    env.run(until=simulation_time)

if __name__ == "__main__":
    main()


Customer 1 served. Waiting time: 0
Customer 2 served. Waiting time: 0
Customer 3 served. Waiting time: 0
Customer 4 served. Waiting time: 0
